# 🐼 Pandas Workshop — Manhattan Housing (Student Edition)

Welcome! This notebook teaches **pandas** step-by-step using a real-world style dataset (we'll call it *Manhattan Housing*).  
It’s designed for learning: every section includes **clear goals** and explains **what each code cell does** in plain language.

> **Before you start:** Put your CSV file in the same folder as this notebook and name it `manhattan_housing.csv`.  
> If your columns have different names (they probably will!), look for **TODO** comments and edit the names accordingly.

### What we'll learn
- Loading data into pandas and getting a first look
- Selecting columns/rows; filtering with conditions
- Handling missing values and fixing data types
- Working with dates (month, year, resampling)
- Feature engineering (e.g., price per square foot)
- Grouping, aggregating, and pivot tables
- Sorting, ranking, and window functions
- Joining/merging with another table
- String operations and categoricals
- Plotting with matplotlib (simple and effective)
- Performance tips
- Practice exercises + worked solutions

---


## 0) Setup

**Goal:** Import the libraries we'll use and check versions so everyone is on the same page.

**What this does:**
- Imports `pandas`, `numpy`, and `matplotlib.pyplot`
- Prints the versions of Python, pandas, and numpy (helpful for debugging)
- Enables inline plotting for charts


In [ ]:
import sys, platform, pandas as pd, numpy as np, matplotlib.pyplot as plt

print('Python:', sys.version.split()[0])
print('Platform:', platform.platform())
print('pandas:', pd.__version__)
print('numpy:', np.__version__)

%matplotlib inline


## 1) Load the data

**Goal:** Read a CSV file into a pandas **DataFrame**.

**What this does:**
- Reads `manhattan_housing.csv` into `df`
- `low_memory=False` makes pandas read types more consistently
- Shows the shape (rows × columns)
- Displays the first 5 rows to get a feel for the data

> **TODO:** If your file isn't named `manhattan_housing.csv`, update the `CSV_PATH` below.


In [ ]:
CSV_PATH = 'manhattan_housing.csv'  # TODO: update if your file has a different name/path
df = pd.read_csv(CSV_PATH, low_memory=False)
print('Rows x Columns:', df.shape)
df.head()


## 2) First look: structure & schema

**Goal:** Understand what columns exist, their data types, and peek at sample rows.

**What this does:**
- Lists column names and data types
- Shows a random sample (to see varied rows)
- Prints `info()` to view non-null counts and dtypes (great for spotting missing values)


In [ ]:
df.columns.tolist(), df.dtypes

In [ ]:
df.sample(min(5, len(df)), random_state=42)

In [ ]:
df.info()

## 3) Selecting columns & rows

**Goal:** Learn how to pick columns and filter rows.

**What this does:**
- Creates a smaller DataFrame `df_small` with columns of interest (edit the list below!)
- Demonstrates `head()`, and basic filtering
- Uses a condition to keep only rows with a positive sale price

> **TODO:** Replace column names in `cols_of_interest` to match your dataset. Examples you might have:  
> `sale_price`, `sale_date`, `neighborhood`, `gross_square_feet`, `land_square_feet`, `year_built`


In [ ]:
# Choose columns you care about (edit these to match your actual CSV)
cols_of_interest = [
    'sale_price',            # TODO
    'sale_date',             # TODO
    'neighborhood',          # TODO
    'gross_square_feet',     # TODO
    'land_square_feet',      # TODO
    'year_built'             # TODO
]

existing_cols = [c for c in cols_of_interest if c in df.columns]
df_small = df[existing_cols].copy() if existing_cols else df.copy()

print('df_small columns:', df_small.columns.tolist())
df_small.head(3)


In [ ]:
# Filter example: keep rows where sale price is positive (change the column name if needed)
if 'sale_price' in df_small.columns:
    df_pos = df_small[df_small['sale_price'] > 0]
    print('Positive-price rows:', df_pos.shape[0])
    df_pos.head()
else:
    print("TODO: Replace 'sale_price' with an existing numeric price column in your data.")


## 4) Cleaning & missing values

**Goal:** See where values are missing and apply simple strategies to handle them.

**What this does:**
- Calculates the fraction of missing values per column
- Creates `df_clean` by filling numeric columns with their median and text columns with their mode (most frequent value)
- Re-checks missingness afterward

> There’s no one-size-fits-all approach—this is just a straightforward starting point.


In [ ]:
df_small.isna().mean().sort_values(ascending=False).head(10)

In [ ]:
df_clean = df_small.copy()
for c in df_clean.columns:
    if pd.api.types.is_numeric_dtype(df_clean[c]):
        df_clean[c] = df_clean[c].fillna(df_clean[c].median())
    else:
        mode_vals = df_clean[c].mode()
        df_clean[c] = df_clean[c].fillna(mode_vals.iloc[0] if len(mode_vals) else df_clean[c])
df_clean.isna().mean().sort_values(ascending=False).head(5)


## 5) Types & datetime handling

**Goal:** Work with dates so we can group by month/year and do time-based analysis.

**What this does:**
- Converts a column to datetime (`sale_date`—change if needed)
- Extracts `sale_year`, `sale_month`, and `sale_quarter` to use later for grouping

> **TODO:** If your date column has a different name, change `'sale_date'` below.


In [ ]:
if 'sale_date' in df_clean.columns:
    df_clean['sale_date'] = pd.to_datetime(df_clean['sale_date'], errors='coerce', infer_datetime_format=True)
    df_clean['sale_year'] = df_clean['sale_date'].dt.year
    df_clean['sale_month'] = df_clean['sale_date'].dt.month
    df_clean['sale_quarter'] = df_clean['sale_date'].dt.to_period('Q')
    df_clean[['sale_date','sale_year','sale_month','sale_quarter']].head()
else:
    print("TODO: Replace 'sale_date' with your timestamp column.")


## 6) Feature engineering

**Goal:** Create useful derived columns for analysis.

**What this does:**
- Computes **price per square foot (ppsf)** from `sale_price / gross_square_feet`
- Protects against division by zero and missing values

> **TODO:** Update the column names if needed.


In [ ]:
import numpy as np

if {'sale_price', 'gross_square_feet'} <= set(df_clean.columns):
    df_clean['ppsf'] = df_clean['sale_price'] / df_clean['gross_square_feet'].replace(0, np.nan)
    df_clean['ppsf'] = df_clean['ppsf'].replace([np.inf, -np.inf], np.nan)
    df_clean[['sale_price','gross_square_feet','ppsf']].head()
else:
    print("TODO: Create a 'ppsf' feature using your price and area columns.")


## 7) Groupby & aggregation

**Goal:** Summarize data by categories (e.g., neighborhood and year).

**What this does:**
- Groups rows by `neighborhood` and `sale_year`
- Calculates count, mean, median, min, max of `sale_price`
- Sorts by mean price to see which groups are highest

> **TODO:** Make sure the grouping and metric columns exist.


In [ ]:
group_cols = [c for c in ['neighborhood', 'sale_year'] if c in df_clean.columns]
if group_cols and 'sale_price' in df_clean.columns:
    g = (df_clean
         .groupby(group_cols, dropna=False)['sale_price']
         .agg(['count','mean','median','min','max'])
         .sort_values('mean', ascending=False)
        )
    g.head(10)
else:
    print("TODO: Adjust 'group_cols' and the target metric for your dataset.")


## 8) Pivot tables

**Goal:** Reshape data to compare values across two dimensions.

**What this does:**
- Creates a pivot table of **median PPSF** by `neighborhood` (rows) and `sale_year` (columns)

> **Tip:** Pivot tables are great for heatmaps or quick comparisons.


In [ ]:
if {'neighborhood', 'sale_year', 'ppsf'} <= set(df_clean.columns):
    pivot_ppsf = pd.pivot_table(
        df_clean,
        index='neighborhood',
        columns='sale_year',
        values='ppsf',
        aggfunc='median'
    )
    pivot_ppsf.head()
else:
    print("TODO: Ensure 'neighborhood', 'sale_year', 'ppsf' exist for the pivot example.")


## 9) Sorting, ranking, and window functions

**Goal:** Order rows and find the "top N" within groups.

**What this does:**
- Sorts the DataFrame by price (descending)
- Ranks sales within each `neighborhood` × `sale_year` and shows the top 3

> **Why this matters:** Window functions help you compare rows within their peer groups.


In [ ]:
if 'sale_price' in df_clean.columns:
    df_sorted = df_clean.sort_values('sale_price', ascending=False)
    df_sorted.head(10)
else:
    print("TODO: Replace 'sale_price' with your target sort column.")


In [ ]:
if {'neighborhood', 'sale_year', 'sale_price'} <= set(df_clean.columns):
    df_clean = df_clean.sort_values(['neighborhood','sale_year','sale_price'], ascending=[True, True, False])
    df_clean['rank_in_nbhd_year'] = df_clean.groupby(['neighborhood','sale_year'])['sale_price'].rank(method='first', ascending=False)
    df_clean[df_clean['rank_in_nbhd_year'] <= 3].head(10)
else:
    print("TODO: Ensure grouping and value columns exist for ranking demo.")


## 10) Joins & merges (optional)

**Goal:** Combine two tables using a shared key.

**What this does:**
- Creates a small "supplemental" table with a `zip_code` and a fake `park_access_score`
- Merges it into the main data on `zip_code` using a left join

> In real projects, you'd load a second CSV and merge on a real key.


In [ ]:
supp = pd.DataFrame({
    'zip_code': [10001, 10002, 10003],
    'park_access_score': [72, 64, 81]
})
if 'zip_code' in df_clean.columns:
    df_merged = df_clean.merge(supp, on='zip_code', how='left')
    df_merged[['zip_code','park_access_score']].head()
else:
    print("Optional: add a ZIP code column to demo merges.")


## 11) String operations & categoricals

**Goal:** Clean text columns and improve performance with categoricals.

**What this does:**
- Standardizes `neighborhood` names (strip whitespace, Title Case)
- Converts some object (string) columns to **category** type to save memory

> **Tip:** Categoricals are great for columns with repeated labels.


In [ ]:
# String cleaning example
if 'neighborhood' in df_clean.columns:
    df_clean['neighborhood'] = df_clean['neighborhood'].astype(str).str.strip().str.title()
    df_clean['neighborhood'].head()
else:
    print("Optional: add a 'neighborhood' string column to demo string ops.")


In [ ]:
# Convert object columns to category (where sensible)
for c in ['neighborhood','borough','building_class_category']:
    if c in df_clean.columns and df_clean[c].dtype == 'object':
        df_clean[c] = df_clean[c].astype('category')
df_clean.info(memory_usage='deep')


## 12) Plotting with matplotlib

**Goal:** Visualize key patterns with simple charts.

**What this does:**
- Plots a histogram of `sale_price`
- Plots a monthly median `sale_price` time series (if a date column was parsed)
- Plots a bar chart of the top 10 neighborhoods by median PPSF

> We use plain matplotlib here. (No seaborn, and we don't set any custom colors.)


In [ ]:
# 12.1 Histogram of sale prices
if 'sale_price' in df_clean.columns:
    plt.figure()
    df_clean['sale_price'].dropna().plot(kind='hist', bins=50, title='Distribution of Sale Price')
else:
    print("TODO: Provide a numeric price column for the histogram.")


In [ ]:
# 12.2 Time series: monthly median price
if {'sale_date','sale_price'} <= set(df_clean.columns):
    monthly = (df_clean
               .set_index('sale_date')
               .resample('MS')['sale_price']
               .median())
    plt.figure()
    monthly.plot(title='Monthly Median Sale Price')
else:
    print("TODO: Ensure 'sale_date' is parsed to datetime for resampling.")


**Interpretation tips:**  
- Histograms show spread and outliers (e.g., unusually high sales).  
- Time series help identify trends or seasonality.  
- Bar charts make categorical comparisons easy (e.g., neighborhoods).

In [ ]:
# 12.3 Bar chart: top neighborhoods by median PPSF
if {'neighborhood','ppsf'} <= set(df_clean.columns):
    nbhd_ppsf = (df_clean
                 .groupby('neighborhood')['ppsf']
                 .median()
                 .sort_values(ascending=False)
                 .head(10))
    plt.figure()
    nbhd_ppsf.plot(kind='bar', title='Top 10 Neighborhoods by Median PPSF')
else:
    print("TODO: Create 'ppsf' and 'neighborhood' columns for this plot.")


## 13) Performance & memory tips

**Goal:** Learn how to keep analyses fast and memory-efficient.

**What this does:**
- Shows memory usage by column (so you can target the biggest ones)
- Demonstrates vectorization vs. `apply` (vectorized operations are usually faster)


In [ ]:
# Memory usage by column (largest first)
mem = df_clean.memory_usage(deep=True).sort_values(ascending=False)
mem.head(10)


In [ ]:
# Vectorization vs. apply micro-benchmark (small demo)
import time
if 'sale_price' in df_clean.columns:
    s = df_clean['sale_price'].fillna(0).head(100000) if len(df_clean) > 0 else pd.Series([])
    start = time.time()
    v = s * 1.05  # vectorized 5% increase
    t_vec = time.time() - start

    start = time.time()
    a = s.apply(lambda x: x * 1.05)
    t_apply = time.time() - start

    print(f"Vectorized: {t_vec:.6f}s | apply: {t_apply:.6f}s (smaller is faster)")
else:
    print("Optional: provide a numeric column to demo vectorization.")


## 14) Exercises (practice)

Try these on your own first. Use the code above as a reference and adjust column names to your dataset.

1. **Data hygiene**:  
   - Drop rows with non‑positive `sale_price`.  
   - Remove outliers: keep only the 1st–99th percentiles of `sale_price`.
2. **Datetime**:  
   - Parse `sale_date` and create `sale_year`, `sale_month`.
3. **Aggregation**:  
   - Compute median `ppsf` by `neighborhood` and list the top 5.
4. **Window functions**:  
   - Within each `neighborhood`, rank sales by `sale_price` and select the top 3 per neighborhood.
5. **Visualization**:  
   - Plot a time series of monthly median `sale_price`.


## 15) Exercise solutions (reveal only after trying)

Below are straightforward solutions for the exercises.


In [ ]:
# E1 solution — Data hygiene
df_sol = df.copy()
if 'sale_price' in df_sol.columns:
    df_sol = df_sol[df_sol['sale_price'] > 0]
    lo, hi = df_sol['sale_price'].quantile([0.01, 0.99])
    df_sol = df_sol[df_sol['sale_price'].between(lo, hi)]
    df_sol.head()
else:
    print("Update 'sale_price' to match your dataset.")


In [ ]:
# E2 solution — Datetime parsing
if 'sale_date' in df_sol.columns:
    df_sol['sale_date'] = pd.to_datetime(df_sol['sale_date'], errors='coerce', infer_datetime_format=True)
    df_sol['sale_year'] = df_sol['sale_date'].dt.year
    df_sol['sale_month'] = df_sol['sale_date'].dt.month
    df_sol[['sale_date','sale_year','sale_month']].head()


In [ ]:
# E3 solution — Aggregation
if {'sale_price','gross_square_feet'} <= set(df_sol.columns):
    df_sol['ppsf'] = df_sol['sale_price'] / df_sol['gross_square_feet'].replace(0, np.nan)
    top5 = (df_sol.groupby('neighborhood')['ppsf']
            .median()
            .sort_values(ascending=False)
            .head(5))
    top5


In [ ]:
# E4 solution — Window functions
if {'neighborhood','sale_price'} <= set(df_sol.columns):
    df_sol = df_sol.sort_values(['neighborhood','sale_price'], ascending=[True, False])
    df_sol['rank_in_nbhd'] = df_sol.groupby('neighborhood')['sale_price'].rank(method='first', ascending=False)
    df_sol[df_sol['rank_in_nbhd'] <= 3].head(10)


In [ ]:
# E5 solution — Visualization
if {'sale_date','sale_price'} <= set(df_sol.columns):
    monthly = (df_sol
               .set_index('sale_date')
               .resample('MS')['sale_price']
               .median())
    plt.figure()
    monthly.plot(title='Monthly Median Sale Price (Solution)')
